# Homework 8
Currently breaking it up into subsections that I can work on

QUESTIONS:

Should you be able to upload database files also??

TODO:
error handling on impromper queries / DONEISH


display collections available / - DONE


clear database /


force reference tag to be unique? Or force some combo of reference tag + collection to be unique? /


display text about how to query / DONE


andd pre-text to query to eliminate the SELECT and such. / DONE


testing, testing, testing /

fix authors, DONE


fix journals?, some sort of type



In [38]:
from flask import Flask, render_template, request, url_for, redirect, flash
import flask
import pybtex.database as pb
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.secret_key = 'Super Secret PW'
db=SQLAlchemy(app)

#TODO: somehow move at least some of this to another file?
class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ref_tag = db.Column(db.String)
    Collection = db.Column(db.String)
    #Adsnote = db.Column(db.String)
    #Adsurl = db.Column(db.String)
    Author = db.Column(db.String)
    #Date_Added = db.Column(db.Date)
    #Date_Modified = db.Column(db.Date)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    #Month = db.Column(db.String)
    Pages = db.Column(db.String) #I think?
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)
    #Bdsk_File_1  = db.Column(db.String)
    
db.create_all()

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [5]:
#test out pybtex
import pybtex.database as pb

def add_entry_to_db(key, entry, collection):
    keywords = {}
    for field in ['Title', 'Author','Journal', 'Keywords', 'Pages', 'Volume', 'Year']:
        if field in entry.fields:
            keywords[field] = entry.fields[field]
        else:
            keywords[field] = None
    db.session.add(Citation(**keywords, Collection = collection, ref_tag = key))
    db.session.commit()
x=pb.parse_file('hw_8_data/homework_8_refs.bib')
for k,v in x.entries.items():
    add_entry_to_db(k, v, 'col')
    
    
x=pb.parse_file('hw_8_data/homework_8_refs.bib')
for k,v in x.entries.items():
    add_entry_to_db(k, v, 'col2')

In [187]:
[c.Title for c in Citation.query.filter(Citation.Year < 1980).all()]

['{Reddenings of Cepheids using BVI photometry}',
 '{1777 variables in the Magellanic Clouds}',
 '{Periods of 25 Variable Stars in the Small Magellanic Cloud.}']

In [233]:
res = db.session.execute('SELECT * FROM bibliography WHERE 3')

# Part 1: Basic Flask Website Architecture

In [79]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
    <head>
        <title>My Super Site</title>
    </head>

    <body>
        <h1>{{ page_title }}</h1>
        
        <h2>
    <a href="{{ url_for('home') }}">Home</a>
    <a href="{{ url_for('query_db') }}">Query Database</a>
    <a href="{{ url_for('upload_file') }}">Upload a File</a>
    </h2>

    {{ message | safe}}        
    {% block content %}{% endblock %}        
    </body>
</html>


Overwriting templates/base.html


In [140]:
%%writefile templates/home.html
{% extends "base.html" %}
{% block content %}
<form action="/home/" method="POST">
            <input type="submit" name = 'Clear_Database' value = 'Clear Database'/>
</form>
{% endblock %}

Overwriting templates/home.html


In [138]:
%%writefile templates/form.html
{% extends "base.html" %}
{% block content %}
<p>
            Query the database using sql query syntax, entering the string that follows the WHERE.
            </br>
            Column names are 'ReferenceTag','Collection', 'Title', 'Author','Journal', 'Keywords',
              'Pages', 'Volume', 'Year' and 'id'.</br>
            To use wildcards, use LIKE and % as a multiple character wildcard and _ as a single
            character wildcard. </br>
            Explicitly put strings in quotes. Note that the pages column is a string.</p>
            <form action="/query_database/" method="POST">
            Make a query: 
            <input type="text" name="query" />
            <input type="submit" value ="Submit Query"/>
            </form>
            {% with messages = get_flashed_messages() %}
            {% if messages %}
    <ul class=flashes>
    {% for message in messages %}
      <li>{{ message | safe}}</li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
{% endblock %}

Overwriting templates/form.html


In [141]:
%%writefile templates/file_up.html
{% extends "base.html" %}

{% set page_title = 'Choose file to upload' %}
{% block content %}
   
      <form action = "/upload_file/" method = "POST" 
         enctype = "multipart/form-data">
            Collection Name:
        <input type = "text" name = "collection_name" />
        File:
         <input type = "file" name = "file" />
         <input type = "submit" value = "Upload File"/>
      </form>
      
{% endblock %}

Overwriting templates/file_up.html


In [85]:
%%writefile config.py

import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI = \
        'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False
    SECRET_KEY = 'Super Secret PASSWORD'
    DEBUG = True

Overwriting config.py


In [102]:
%%writefile basic_site.py

from flask import Flask, render_template, request, url_for, redirect, flash
import flask
import pybtex.database as pb
from flask_sqlalchemy import SQLAlchemy
from config import Config
import numpy as np
from database import db, Citation
from sqlalchemy.exc import OperationalError

app = Flask(__name__)
app.config.from_object(Config)
with app.app_context():
    db.init_app(app)
    db.create_all()

@app.route("/")
@app.route("/home")
@app.route("/home/", methods=['GET', 'POST'])
def home():
    """Welcomes the user and tells them how many collections are in the database."""
    if request.method == 'POST':
        with app.app_context():
            db.drop_all()
            db.create_all()
    if Citation.query.first() is None: #check for empty db
        this_message = 'You have no collections. Upload a file to get started!'
    else: #get number of collections and names
        collections = np.unique(Citation.query.with_entities(Citation.Collection).all())
        colls_w_sizes = [s+' (%s entries)' %get_size_collection(s) for s in collections]
        this_message = ('You have %s collections, with names : ' % len(collections)) + \
                        ', '.join([s for s in colls_w_sizes]) 
    
    return render_template('home.html', page_title="Home",
                           message="Welcome! </p>This is a site for querying bibtex files </p>"+
                                   this_message)

@app.route('/query_database/', methods=['GET', 'POST'])
def query_db():
    if request.method == 'POST':
        query = request.form['query']
        #given a query, add the sql prefix stuff to it
        query = 'SELECT * FROM bibliography WHERE ' + query
        if query not in (""," "):
            try:
                result=db.engine.execute(query).fetchall()
            except OperationalError:
                return render_template('form.html', page_title= 'Query Database', 
                                   message='Invalid query. Please try again.')
            if len(result) > 0:
                display_query_results(result)
            else:
                flash('No results to display :(')
            return render_template('form.html', page_title= 'Results for : ' + query)
        else:
            return render_template('form.html', page_title= 'Query Database', 
                                   message='Invalid query. Please try again')
    else:
        ## this is a normal GET request
        return render_template("form.html", page_title = 'Query a Database :)')
    
@app.route('/upload_file/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template('file_up.html', message = 'Please Choose a File')
        file = request.files['file']
        coll_name = request.form['collection_name']
        if coll_name not in (""," "):
            add_file_to_db(coll_name, file)
            return render_template('base.html', page_title= coll_name)
        else:
            return render_template('file_up.html',
                                   message = 'Please enter a valid collection name')
    else:
        ## this is a normal GET request
        return render_template("file_up.html", page_title = 'Upload a File :)')

def display_query_results(results):
    fields = ['ReferenceTag','Collection', 'Title', 'Author','Journal', 'Keywords',
              'Pages', 'Volume', 'Year']
    for row in results:
        for field in fields:
            if field == 'Year':
                flash(field + ': ' + str(row[field]) + '</br></br>')
            else:
                flash(field + ': ' + str(row[field]))
                
def add_file_to_db(coll_name, file):
    parsed_file=pb.parse_bytes(file.read(), bib_format='bibtex')
    for k, entry in parsed_file.entries.items():
        add_entry_to_db(k, entry, coll_name)
                
def add_entry_to_db(key, entry, collection):
    keywords = {}
    for field in ['Title', 'Journal', 'Keywords', 'Pages', 'Volume', 'Year']:
        if field in entry.fields:
            keywords[field] = format_btex_fields(entry.fields[field])
        else:
            keywords[field] = None
    #handle author separately because its stored differently
    if 'Author' in entry.persons:
        author_list = [format_btex_fields(str(auth)) for auth in entry.persons['Author']]
        keywords['Author'] = ', '.join(author_list)
    else:
        keywords['Author'] = None
    with app.app_context():
        db.session.add(Citation(**keywords, Collection = collection, ReferenceTag = key))
        db.session.commit()
    
def format_btex_fields(string):
    """helper method to remove odd formatting from btex files"""
    string = string.replace('{','')
    string = string.replace('}','')
    return string

def get_size_collection(collection):
    """helper method to get the number of entries in a collection"""
    return len(Citation.query.filter(Citation.Collection == collection).all())

if __name__ == "__main__":
    #setup_app()
    app.run()

Overwriting basic_site.py


In [103]:
%%writefile database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ReferenceTag = db.Column(db.String)
    Collection = db.Column(db.String)
    Author = db.Column(db.String)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    Pages = db.Column(db.String)
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)


Overwriting database.py


In [99]:
%run basic_site.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


# Part 2: Add travis testing to Basic Flask

In [106]:
%%writefile testing_test.py
import os
import unittest
from basic_site import app, db, Citation, add_file_to_db
from io import BytesIO
from flask_sqlalchemy import SQLAlchemy

class FlaskTestCase(unittest.TestCase):

    def setUp(self):
        self.app = app.test_client()
        with app.app_context():
            db.drop_all()
            db.create_all()
    
    def test_home_page(self):
        rv = self.app.get('/')
        assert b'Welcome!' in rv.data
    
    def submit_query(self, query):
        #helper function to test querying
        return self.app.post('/query_database/', data=dict(
            query=query
        ), follow_redirects=True)
    
    def test_query_page(self):
        rv = self.app.get('/query_database/')
        assert b'name="query"' in rv.data
    
    def test_upload_file_page(self):
        rv = self.app.get('/upload_file/')
        
        assert b'Choose file to upload' in rv.data
        assert b'Upload a File' in rv.data
    
    def upload_file(self, coll_name, file, filename):
        """helper method to mimic the user choosing a file and collection name"""
        data = {
            'collection_name': coll_name,
            'file': (file, filename)
        }

        return self.app.post('/upload_file/', buffered=True,
                         content_type='multipart/form-data',
                         data=data)
        
    def test_file_upload(self):
        rv = self.upload_file('ex', open('hw_8_data/homework_8_refs.bib','rb'),
                              'hw_8_data/homework_8_refs.bib')
        assert rv.status_code == 200
        assert b'ex' in rv.data
        with app.app_context():
            assert len(Citation.query.all()) > 0
    
    def test_query_submission(self):
        """tests invalid queries and queries to empty databases"""
        assert b':(' in self.submit_query('Year < 1930').data
        assert b'Invalid' in self.submit_query('').data
        assert b'Invalid' in self.submit_query(' ').data
    
    def test_bad_file_upload(self):
        """tests failure to provide collection names or to choose a file"""
        rv = self.upload_file('',open('hw_8_data/homework_8_refs.bib','rb'),
                              'hw_8_data/homework_8_refs.bib')
        assert b'enter a valid collection name' in rv.data        
        data = {'collection_name': 'ex' }

        rv = self.app.post('/upload_file/', buffered=True,
                         content_type='multipart/form-data',
                         data=data)
        assert b'Please Choose a File' in rv.data

    def test_homepage_text(self):
        """test that the homepage properly displays collection info"""
        rv = self.app.get('/home/')
        assert b'no collections' in rv.data
    
    def test_homepage_text_not_empty(self):
        #then add collections and test that it shows them and entries
        add_file_to_db('ex', open('hw_8_data/homework_8_refs.bib','rb'))
        add_file_to_db('ex2', open('hw_8_data/homework_8_refs.bib','rb'))
        rv = self.app.get('/home/')
        assert b'ex (46 entries)' in rv.data
        assert b'ex2 (46 entries)' in rv.data
        
    def test_real_query(self):
        add_file_to_db('ex', open('hw_8_data/homework_8_refs.bib','rb'))
        rv = self.submit_query('Author LIKE "%Dean%"')
        assert b'Dean' in rv.data
        assert b'Reddenings of Cepheids' in rv.data

if __name__ == '__main__':
    unittest.main()

Overwriting testing_test.py


In [ ]:
!python testing_test.py

....testing_test.py:79: ResourceWarning: unclosed file <_io.BufferedReader name='hw_8_data/homework_8_refs.bib'>
  add_file_to_db('ex', open('hw_8_data/homework_8_refs.bib','rb'))
testing_test.py:80: ResourceWarning: unclosed file <_io.BufferedReader name='hw_8_data/homework_8_refs.bib'>
  add_file_to_db('ex2', open('hw_8_data/homework_8_refs.bib','rb'))
...